In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [4]:
import numpy as np
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import wav2vec2, bert, fastspeech, ctc






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
class Encoder:
    def __init__(self, config):
        self.config = config
        self.model = None
    def __call__(self, x, input_mask, training = True):
        if self.model is None:
            input_mask = tf.logical_not(input_mask)
            self.model = bert.BertModel(config = self.config, is_training = training,
                                  input_ids = x, input_mask = input_mask)
        return self.model.sequence_output
    
encoder = Encoder(config = bert.BertConfig())

In [6]:
cfg = wav2vec2.Wav2Vec2Config(mask_prob = 0.5, mask_channel_prob = 0.1,
                             mask_channel_length = 64,
                             feature_grad_mult = 0.0)
model = wav2vec2.Model(cfg, encoder)

In [7]:
X = tf.placeholder(tf.float32, (None, None))
X_len = tf.placeholder(tf.int32, (None,))
Y = tf.placeholder(tf.int32, (None, None))

In [8]:
r = model(X, padding_mask = X_len, features_only = True)
r

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


{'x': <tf.Tensor 'wav2vec2/bert/encoder/Reshape_13:0' shape=(?, ?, 768) dtype=float32>,
 'padding_mask': <tf.Tensor 'wav2vec2/LogicalNot:0' shape=(?, ?) dtype=bool>}

In [9]:
logits = tf.layers.dense(r['x'], 100 + 1)
seq_lens = tf.reduce_sum(tf.cast(tf.logical_not(r['padding_mask']), tf.int32), axis = 1)
targets_int32 = tf.cast(Y, tf.int32)
mean_error, sum_error, sum_weight = ctc.loss.ctc_loss(
    logits, targets_int32, seq_lens
)
loss = mean_error

Instructions for updating:
Use `tf.cast` instead.



In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
y, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav')
len(y) / sr

3.518625

In [12]:
y = np.zeros((sr * 17,))

In [13]:
%%time

o = sess.run(r, feed_dict = {X: [y, y], X_len: [len(y), len(y) - 10000]})
o['x'].shape

CPU times: user 20.3 s, sys: 3.56 s, total: 23.9 s
Wall time: 4.73 s


(2, 849, 768)

In [14]:
%%time

sess.run(loss, feed_dict = {X: [y], X_len: [len(y)],
                               Y: [[1,2,3,4,5,6]]})

CPU times: user 9.61 s, sys: 1.32 s, total: 10.9 s
Wall time: 2.53 s


3693.1091

In [15]:
tf.trainable_variables()

[<tf.Variable 'vars:0' shape=(1, 640, 128) dtype=float32_ref>,
 <tf.Variable 'mask_emb:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'wav2vec2/ConvFeatureExtractionModel/sequential/conv1d/kernel:0' shape=(10, 1, 512) dtype=float32>,
 <tf.Variable 'wav2vec2/ConvFeatureExtractionModel/sequential/group_normalization/gamma:0' shape=(512,) dtype=float32>,
 <tf.Variable 'wav2vec2/ConvFeatureExtractionModel/sequential/group_normalization/beta:0' shape=(512,) dtype=float32>,
 <tf.Variable 'wav2vec2/ConvFeatureExtractionModel/sequential_1/conv1d_1/kernel:0' shape=(3, 512, 512) dtype=float32>,
 <tf.Variable 'wav2vec2/ConvFeatureExtractionModel/sequential_2/conv1d_2/kernel:0' shape=(3, 512, 512) dtype=float32>,
 <tf.Variable 'wav2vec2/ConvFeatureExtractionModel/sequential_3/conv1d_3/kernel:0' shape=(3, 512, 512) dtype=float32>,
 <tf.Variable 'wav2vec2/ConvFeatureExtractionModel/sequential_4/conv1d_4/kernel:0' shape=(3, 512, 512) dtype=float32>,
 <tf.Variable 'wav2vec2/ConvFeatureExtractionMod